In [5]:
import pandas as pd
import glob
import json
#import twitter_col
import re
from tqdm import tqdm
import pickle
import numpy as np
from datetime import datetime
from datetime import date

In [7]:
#Part 2. Temporal Topic Modeling
#This section parses tweets associated to Chinese Accounts: Original or Retweets

In [37]:
#1) Filter out only Chinese-State-Sponsored Tweets ~7,419k tweets  --> json 
input_folder = 'Data/final_data/Taiwan/'

names = []
messages = []
dates = []
ids = []
result = []

total_tweets = 0 

for input_file in glob.glob("{}*.json".format(input_folder)):
    with open(input_file, "r", encoding='utf-8') as f:
        for index, line in enumerate(f):
            full_dict = json.loads(line)
            total_tweets +=1
            if full_dict['lang'] == 'en':
    
                
                try:
                    name = full_dict['user']['screen_name']
                    user_id = full_dict['user']['id']
                    date = full_dict['created_at']
                    text = full_dict['text']
                    #mentions only used if you need the retweets also 
                    if full_dict['entities']['user_mentions'][0]['screen_name']:
                        mention = full_dict['entities']['user_mentions'][0]['screen_name']
                    else: 
                        mention = 'None'
                        print ('No mentions')
                    

                    names.append(name)
                    dates.append(date)
                    ids.append(user_id)
                    messages.append(text)
                    result.append(full_dict)
         
                except: 
                    pass
                    #print ('unable to grab screen name')       
                    
print ("total tweets", total_tweets)
filename = input_folder + '/State-Sponsored-Tweets/Taiwan-Tweets-Retweets-All-JAN23.json'
# print("    Writing " + str(len(result)) + " tweets to file: " + filename)
# with open(filename, 'w') as outfile:
#     for tweet in result:
#         print(json.dumps(tweet, sort_keys=True), file=outfile)

total tweets 4255560
    Writing 155836 tweets to file: Data/State-Sponsored-Tweets/Taiwan-Tweets-Retweets-All-Jan23.json


In [3]:
#split all Twitter-Retweet data into 3 period
pelosi1 = []
pelosi2= []
pelosi3 = []


with open("Data/State-Sponsored-Tweets/Taiwan-Tweets-Retweets-All-Jan23.json", "r", encoding='utf-8') as f:
    for index, line in enumerate(f):
        full_dict = json.loads(line)
        tweet_date = datetime.strptime(full_dict['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        #print (tweet_date)
        if tweet_date > datetime.strptime("2022-04-07", '%Y-%m-%d') and tweet_date < datetime.strptime("2022-08-02", '%Y-%m-%d'):
             pelosi1.append(full_dict)
        if tweet_date > datetime.strptime("2022-08-02", '%Y-%m-%d') and tweet_date < datetime.strptime("2022-08-09", '%Y-%m-%d'):
             pelosi2.append(full_dict)
        elif tweet_date > datetime.strptime("2022-08-09", '%Y-%m-%d') and tweet_date < datetime.strptime("2022-10-09", '%Y-%m-%d'):
             pelosi3.append(full_dict)
                

In [4]:
filename = 'Data/State-Sponsored-Tweets/pre-pelosi2.json'
print("    Writing " + str(len(pelosi1)) + " tweets to file: " + filename)
with open(filename, 'w') as outfile:
    for tweet in pelosi1:
        print(json.dumps(tweet, sort_keys=True), file=outfile)
        
filename = 'Data/State-Sponsored-Tweets/mid-pelosi2.json'
print("    Writing " + str(len(pelosi2)) + " tweets to file: " + filename)
with open(filename, 'w') as outfile:
    for tweet in pelosi2:
        print(json.dumps(tweet, sort_keys=True), file=outfile)
        
filename = 'Data/State-Sponsored-Tweets/post-pelosi2.json'
print("    Writing " + str(len(pelosi3)) + " tweets to file: " + filename)
with open(filename, 'w') as outfile:
    for tweet in pelosi3:
        print(json.dumps(tweet, sort_keys=True), file=outfile)

    Writing 12734 tweets to file: Data/State-Sponsored-Tweets/pre-pelosi2.json
    Writing 109469 tweets to file: Data/State-Sponsored-Tweets/mid-pelosi2.json
    Writing 19331 tweets to file: Data/State-Sponsored-Tweets/post-pelosi2.json


In [38]:
# Put tweets into csv form for Topic Modeling and Regression Modeling

tweets = pd.DataFrame()
tweets['Screen_Name'] = names
tweets['ID'] = ids
tweets['Messages'] = messages
tweets['Date_Stamp'] = dates
tweets.shape

(155836, 4)

In [41]:
from dateutil.parser import parse
china_tweets['Date'] = china_tweets['Date_Stamp'].apply(lambda x: parse(x))

In [42]:
tweets.head()

,Screen_Name,ID,Messages,Date_Stamp,State_Sponsor,probability,IS_BOT,Date
0,BabyCha93005924,1466675766909759493,"RT @CGMeifangZhang: More than 3,000 members of...",Sun Aug 28 23:55:29 +0000 2022,0,0.759997,1,2022-08-28 23:55:29+00:00
1,ClemFandango987,1469045313209966593,"RT @CGMeifangZhang: More than 3,000 members of...",Sun Aug 28 23:33:35 +0000 2022,0,0.784597,1,2022-08-28 23:33:35+00:00
2,ShaotsingFromCN,84245889,"RT @CGMeifangZhang: More than 3,000 members of...",Sun Aug 28 23:31:23 +0000 2022,0,0.520254,0,2022-08-28 23:31:23+00:00
3,jml34577122,1455816305622220800,RT @PDChina: The Eastern Theater Command of th...,Sun Aug 28 23:26:41 +0000 2022,0,0.879624,1,2022-08-28 23:26:41+00:00
4,DWZ777,1426598677280346119,@AndyBxxx @MarshaBlackburn #Taiwan 🇹🇼 is a gre...,Sun Aug 28 23:09:26 +0000 2022,0,0.926665,1,2022-08-28 23:09:26+00:00


In [29]:
tweets.Messages = tweets.apply(lambda row: re.sub(r"http\S+", "", row.Messages).lower(), 1)
tweets.Messages = tweets.apply(lambda row: re.sub('rt', ' ', row.Messages).lower(), 1)
tweets.Messages = tweets.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.Messages.split())), 1)
tweets.Messages = tweets.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.Messages).split()), 1)

In [30]:
china_tweets.head()

,Screen_Name,ID,Messages,Date_Stamp,State_Sponsor,probability,IS_BOT,Date
0,CGMeifangZhang,1222894323152195586,leveraging taiwan to grab the global spotlight...,Sun Aug 28 21:03:52 +0000 2022,1,0.172702,0,2022-08-28 21:03:52+00:00
1,CGMeifangZhang,1222894323152195586,more than members of the global chinese commun...,Sun Aug 28 20:19:17 +0000 2022,1,0.172702,0,2022-08-28 20:19:17+00:00
2,CGMeifangZhang,1222894323152195586,so if the us navy wants to deter the pla saili...,Sun Aug 28 18:40:27 +0000 2022,1,0.172702,0,2022-08-28 18:40:27+00:00
3,MahuiChina,1187372311604858882,he and his son were strongly opposed to taiwan...,Sun Aug 28 17:00:37 +0000 2022,1,0.214706,0,2022-08-28 17:00:37+00:00
4,globaltimesnews,49616273,so if the us navy wants to deter the pla saili...,Sun Aug 28 14:29:56 +0000 2022,1,0.371962,0,2022-08-28 14:29:56+00:00


In [31]:
#Create List structures for topic modeling: Text and Dates
dates = []
data = []
for message in tweets['Messages'].unique():
    data.append(message)
    temp =  china_tweets[china_tweets['Messages'] == message]
    dates.append(temp['Date_Stamp'].to_list()[0])

In [32]:
#Ensure length is the same
len(data)

5074

In [33]:
len(dates)

5074

In [34]:
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - removes any single character tokens
    Parameters
    ----------
        message : The text message to be preprocessed
    Returns
    -------
        text: The preprocessed text
    """ 
    # Lowercase the twit message
    text = message.lower()
    # Replace URLs with a space in the message
    #text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    #text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    #text = re.sub('\@[a-zA-Z0-9]*', ' ', text)
    # Replace everything not a letter or apostrophe with a space
    #text = re.sub('[^a-zA-Z\']', ' ', text)
    text = re.sub('rt', ' ', text)
    text = re.sub('@*:', ' ', text)
    # Remove single letter words
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    
    return text
        
# Process for all messages
preprocessed = [preprocess(message) for message in tqdm(data)]

100%|██████████| 5074/5074 [00:00<00:00, 121953.71it/s]


In [35]:
with open('taiwan_data_no-retweets-2023.pkl', 'wb') as f:
    pickle.dump(data, f)